In [ ]:
# Load numpy and scipy integration packages

import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import math

# We're going to need to import scipy.fft packages to analyze the dispersion relation


In [ ]:
class Particle_in_cell_2D:
        """
        ##############################################
        This is a program of 2D PIC (Particle In Cell)
        ##############################################
        """

        """
        1. Do not use n1,n2,n3... in this code to be a variable name.
        """
    def __init__(self, Lx, Lz, Nx, Nz, t_max, dt, dt_out = 0.05, alpha=1, D=1):

        ###########################################################
        # Resolution Parameters
        ###########################################################
        self.Lx = Lx # Length of box in x-axis
        self.Lz = Lz # Length of box in z-axis
        self.s = 3*np.pi # Shearing velocity

        self.Nx = Nx # Number of grids in x-direction
        self.Nz = Nz # Number of grids in z-direction
        self.Np = Nx * Nz# Total number of particles of system

        self.dx = Lx / Nx # Step size of x
        self.dz = Lz / Nz # Step size of z

        self.t_max = t_max # Maximun time
        self.dt = dt # Time difference for one time step
        self.t_span = int(t_max/self.dt) # Max number of steps in t
        self.dt_out = dt_out # Initialized output time point (For example every 0.5 of t output once)

        ##########################################################
        # Scaling parameters
        ##########################################################
        self.c_Va = 20 # Ratio between speed of light and Alfven speed
        self.q_sign = -1 #Sets charge of particles to electrons. Needs to be changed for more complicated code.
        self.omega_ci_by_0 = 11 #Normalized 1/omega0
        self.mi_by_me = 10 # ratio between mass of electron and ion

        self.Normalize =  2 * np.pi * self.q_sign * self.omega_ci_by_0 * self.mi_by_me #Front factor term in U calculations

        #Smoothing factors
        self.alpha = alpha
        self.D = D

        #########################################################
        # Grid Initializations
        ########################################################
        # Set positions of particles in middle of grid
        temp_x = np.arange(0.5*self.dx,Lx+0.5*self.dx,self.dx)
        temp_z = np.arange(0.5*self.dz,Lz+0.5*self.dz,self.dz)

        #Creates 2D array to flatten later
        self.position_x = np.zeros((Nx,Nz))
        self.position_z = np.zeros((Nx,Nz))

        for ix in range(Nx):
            self.position_x[ix,:] = temp_x[ix]

        for iz in range(Nz):
            self.position_z[:,iz] = temp_z[iz]

        #Should represent x and z positions of particles
        self.position_x = self.position_x.flatten()
        self.position_z = self.position_z.flatten()

        # Index representation of particle positions above
        temp_x_index = np.zeros(Nx)
        self.grid_x = np.zeros((Nx,Nz)) # Set the specific x-index of particle in grid

        temp_z_index = np.zeros(Nz)
        self.grid_z = np.zeros((Nx,Nz)) # Set the specific z-index of particle in grid

        #Same as above, 2D array to flatten
        for ix in range(Nx):
            temp_x_index[ix] = int(ix)

        for iz in range(Nz):
            temp_z_index[iz] = int(iz)

        for ix in range(Nx):
            self.grid_x[ix,:] = temp_x_index[ix]

        for iz in range(Nz):
            self.grid_z[:,iz] = temp_z_index[iz]

        self.grid_x = self.grid_x.flatten()
        self.grid_z = self.grid_z.flatten()

        self.N_grid = np.ones((self.Nx, self.Nz)) # Grid that restore number of particles


        # Initializes fields for output:
        self.Bx_stored = []
        self.By_stored = []
        self.Bz_stored = []

        self.Ex_stored = []
        self.Ey_stored = []
        self.Ez_stored = []

        self.Ux_stored = []
        self.Uy_stored = []
        self.Uz_stored = []
        # Initializes positions for output
        self.position_x_stored = []
        self.position_z_stored = []

# Define a function to expand matrix as we need to add boundary conditions
# We use the vstack and hstack to make the boundary conditions to be period
    def expand_matrix(self, my_matrix):
        new_matrix = np.vstack((my_matrix,my_matrix[0,:]))
        new_matrix = np.vstack((my_matrix[-1,:],new_matrix))
        final_matrix = np.hstack((new_matrix,new_matrix[:,0].reshape(-1,1)))
        final_matrix = np.hstack((new_matrix[:,-1].reshape(-1,1),final_matrix))
        return final_matrix

# Set a function to find the field of grid in one time step
    def find_field_of_grid(self, Bx_old, By_old, Bz_old, Ex_old, Ey_old, Ez_old,
                          Ux_old, Uy_old, Uz_old, Bx_now, By_now, Bz_now, Ex_now, Ey_now,
                           Ez_now, Ux_now, Uy_now, Uz_now):
        # ix and iz are index of the specific grid
        # For input of this function,"Old" term represent the i-1 time condition, "Now"
        # means i time condition, this function will ouput i+i time condition.

        # Expand term to fit the period boundary condition:
        Bx_old1 = self.expand_matrix(Bx_old)
        By_old1 = self.expand_matrix(By_old)
        Bz_old1 = self.expand_matrix(Bz_old)
        Ex_old1 = self.expand_matrix(Ex_old)
        Ey_old1 = self.expand_matrix(Ey_old)
        Ez_old1 = self.expand_matrix(Ez_old)
        Ux_old1 = self.expand_matrix(Ux_old)
        Uy_old1 = self.expand_matrix(Uy_old)
        Uz_old1 = self.expand_matrix(Uz_old)

        Bx_now1 = self.expand_matrix(Bx_now)
        By_now1 = self.expand_matrix(By_now)
        Bz_now1 = self.expand_matrix(Bz_now)
        Ex_now1 = self.expand_matrix(Ex_now)
        Ey_now1 = self.expand_matrix(Ey_now)
        Ez_now1 = self.expand_matrix(Ez_now)
        Ux_now1 = self.expand_matrix(Ux_now)
        Uy_now1 = self.expand_matrix(Uy_now)
        Uz_now1 = self.expand_matrix(Uz_now)

# here is a line of code that make the code above more beautiful:
        #big_list = [Bx_old,Bx_old, By_old, Bz_old, Ex_old, Ey_old, Ez_old,
        #            Ux_old, Uy_old, Uz_old, Bx_now, By_now, Bz_now, Ex_now, Ey_now,
        #            Ez_now, Ux_now, Uy_now, Uz_now]
        #new_big_list = []
        #for i,your_list in enumerate(big_list):
        #    new_big_list[i] = self.expand_matrix(your_list)

        # Set three main step sizes
        dt = self.dt
        dz = self.dz
        dx = self.dx

        #########################################################
        # Translate our equations to code (in matrix calculation)
        #########################################################
        # J:
        J_factor = 0.5 * self.omega_ci_by_0 * self.c_Va * self.q_sign
        Jx = J_factor * self.N_grid * Ux_now1[1:-1,1:-1]
        Jy = J_factor * self.N_grid * Uy_now1[1:-1,1:-1]
        Jz = J_factor * self.N_grid * Uz_now1[1:-1,1:-1]

        # B field equations:
        Bx = Bx_old1[1:-1,1:-1] + self.c_Va * (dt/dz) * (Ey_now1[1:-1,2:] - Ey_now1[1:-1,:-2])
        By = By_old1[1:-1,1:-1] + self.c_Va * (dt/dx) * (Ez_now1[2:,1:-1] - Ez_now1[:-2,1:-1])
        By += - self.c_Va * (dt/dz) * (Ex_now1[1:-1,2:] - Ex_now1[1:-1,:-2]) - 2 *dt *self.s * Bx_now1[1:-1,1:-1]
        Bz = Bz_old1[1:-1,1:-1] - self.c_Va * (dt/dx) * (Ey_now1[2:,1:-1] - Ey_now1[:-2,1:-1])

        # E field equations:
        Ex = Ex_old1[1:-1,1:-1] - self.c_Va * (dt/dz) * (By_now1[1:-1,2:] - By_now1[1:-1,:-2]) - 2 * dt * 4* np.pi * Jx
        Ey = Ey_old1[1:-1,1:-1] - self.c_Va * (dt/dx) * (Bz_now1[2:,1:-1] - Bz_now1[:-2,1:-1]) + self.c_Va * (dt/dz) * (Bx_now1[1:-1,2:] - Bx_now1[1:-1,:-2])
        Ey += - 2 * dt * self.s * Ex_now1[1:-1,1:-1] - 2 * dt * 4 * np.pi * Jy
        Ez = Ez_old1[1:-1,1:-1] + self.c_Va * (dt/dx) * (By_now1[2:,1:-1] - By_now1[:-2,1:-1]) - 2 * dt * 4 * np.pi * Jz

        # U field equations:
        Ux = Ux_old1[1:-1,1:-1] + 2 * dt * 4 * np.pi * Uy_now1[1:-1,1:-1]
        Ux += 2 * dt * self.Normalize * (self.c_Va * Ex_now1[1:-1,1:-1] + (Uy_now1[1:-1,1:-1] * Bz_now1[1:-1,1:-1] - Uz_now1[1:-1,1:-1] * By_now1[1:-1,1:-1]))
        Uy = Uy_old1[1:-1,1:-1] - 2 * dt * np.pi * Ux_now1[1:-1,1:-1]
        Uy += 2 * dt * self.Normalize * (self.c_Va * Ey_now1[1:-1,1:-1] - (Ux_now1[1:-1,1:-1] * Bz_now1[1:-1,1:-1] - Uz_now1[1:-1,1:-1] * Bx_now1[1:-1,1:-1]))
        Uz = Uz_old1[1:-1,1:-1]
        Uz += 2 * dt * self.Normalize * (self.c_Va * Ez_now1[1:-1,1:-1] + (Ux_now1[1:-1,1:-1] * By_now1[1:-1,1:-1] - Uy_now1[1:-1,1:-1] * Bx_now1[1:-1,1:-1]))

        # Call smoothing function on all fields
        Bx = self.alpha*Bx + (1.0-self.alpha)*self.smoothing(Bx)
        By = self.alpha*By + (1.0-self.alpha)*self.smoothing(By)
        Bz = self.alpha*Bz + (1.0-self.alpha)*self.smoothing(Bz)

        Ex = self.alpha*Ex + (1.0-self.alpha)*self.smoothing(Ex)
        Ey = self.alpha*Ey + (1.0-self.alpha)*self.smoothing(Ey)
        Ez = self.alpha*Ez + (1.0-self.alpha)*self.smoothing(Ez)

        Ux = self.alpha*Ux + (1.0-self.alpha)*self.smoothing(Ux)
        Uy = self.alpha*Uy + (1.0-self.alpha)*self.smoothing(Uy)
        Uz = self.alpha*Uz + (1.0-self.alpha)*self.smoothing(Uz)

        # Adaptive steps
        # When our particle move too fast, we make our time step smaller by a factor of 2 with recursion
        if( self.is_too_fast(Ux,Uz) ):
            Bx,By,Bz,Ex,Ey,Ez,Ux,Uy,Uz \
            = self.find_field_of_grid(Bx_old,By_old,Bz_old,Ex_old,Ey_old,Ez_old,Ux_old,Uy_old,Uz_old,
                                  Bx_now,By_now,Bz_now,Ex_now,Ey_now,Ez_now,Ux_now,Uy_now,Uz_now)


        # Return smoothed fields
        return  Bx, By, Bz, Ex, Ey, Ez, Ux, Uy, Uz
    #################################
    # Smoothing function with Laplace
    #################################
    def smoothing(self,field):
        field1 = self.expand_matrix(field)

        new_field = np.zeros((self.Nx,self.Nz))

        for ix in range(1,self.Nx+1):
            for iz in range(1,self.Nz+1):
                new_field[ix-1,iz-1] = self.D * 1.0/(self.dx*self.dx)*(field1[ix+1,iz] + field1[ix-1,iz] - 2*field1[ix,iz])
                new_field[ix-1,iz-1] = new_field[ix-1,iz-1] + self.D * 1.0/(self.dz*self.dz)*(field1[ix,iz+1] + field1[ix,iz-1] - 2*field1[ix,iz])

        return new_field
    # Define a equation to test if our particle is moving too fast
    def is_too_fast(self,Ux,Uz):
        ux_max = np.max(np.absolute(Ux))
        uz_max = np.max(np.absolute(Uz))

        max_x_change = ux_max*self.dt
        max_z_change = uz_max*self.dt

        if max_x_change >= self.dx or max_z_change >= self.dz:
            self.dt = self.dt * 0.5
        # Print the new time step to see if the time step is too small to run
        # This print can also check if our data explode
            print('new dt: ' + str(self.dt))
            return True
        else:
            return False

# Set a function to find the position of
    def find_new_position(self, Ux, Uz):
        # The input Ux and Uz are (Nx * Nz) matrix that corresponding to each grid.
        # If there is boundary condition, use Ux[1:-2,1:-2] to make size of matrix right.
        for n1 in range (self.Np):
            # Index of x and a tell the position of particle in grid.
            index_x = int(self.grid_x[n1])
            index_z = int(self.grid_z[n1])
            # position_{x/z} tell the exact position of a particle,
            # and all the particle follow period boundary condition
            self.position_x[n1] += self.dt * Ux[index_x, index_z]
            self.position_z[n1] += self.dt * Uz[index_x, index_z]
            self.position_x[n1] = self.position_x[n1] % self.Lx
            self.position_z[n1] = self.position_z[n1] % self.Lz

# Set a function to find the distribution of particles in grid
    def re_distribution_on_grid(self):
    # This function will find the new index of x and z of particle
    # Also, this function will count how many particles are in each grid
        for n2 in range(self.Np):
            old_index_x = int(self.grid_x[n2])
            old_index_z = int(self.grid_z[n2])
        # Index of particle in grid follow the period boundary condition
            new_index_x = math.ceil(self.position_x[n2]/self.dx) % self.Nx
            new_index_z = math.ceil(self.position_z[n2]/self.dz) % self.Nz
            self.N_grid[old_index_x, old_index_z] -= 1
            self.N_grid[new_index_x, new_index_z] += 1
            self.grid_x[n2] = new_index_x
            self.grid_z[n2] = new_index_z
    ##################################################
    # Define a function to store the result of field
    ##################################################
    def store_fields(self,Bx,By,Bz,Ex,Ey,Ez,Ux,Uy,Uz):
        # Storing all fields
        self.Bx_stored.append(Bx)
        self.By_stored.append(By)
        self.Bz_stored.append(Bz)

        self.Ex_stored.append(Ex)
        self.Ey_stored.append(Ey)
        self.Ez_stored.append(Ez)

        self.Ux_stored.append(Ux)
        self.Uy_stored.append(Uy)
        self.Uz_stored.append(Uz)

        # Storing particle positions which will also output
        posi_x = np.copy(self.position_x)
        posi_z = np.copy(self.position_z)
        self.position_x_stored.append(posi_x)
        self.position_z_stored.append(posi_z)


    def solve(self):

        # Initialize fields and times variables:
        t_span = self.t_span # Define number of iterations to loop from time
        next_out = self.dt_out # When to save fields next
        print(t_span)
        time = 0 # set a count of time
        # Set the initial values of our problem
        B_z0 = 1
        Bx = np.zeros((self.Nx,self.Nz))
        By = np.zeros((self.Nx,self.Nz))
        Bz = np.zeros((self.Nx,self.Nz))
        Bz[:,:] = B_z0

        Ex = np.zeros((self.Nx,self.Nz))
        Ey = np.zeros((self.Nx,self.Nz))
        Ez = np.zeros((self.Nx,self.Nz))

        Ux = np.zeros((self.Nx,self.Nz))
        Uy = np.zeros((self.Nx,self.Nz))
        Uz = np.zeros((self.Nx,self.Nz))

        for n4 in range(self.Nz):
            Ux[:,n4] = 1/20 * np.sin((2 * np.pi * self.dz * n4/ self.Lz))


        # Stores field initial conditions
        self.store_fields(Bx,By,Bz,Ex,Ey,Ez,Ux,Uy,Uz)

        # Solve for field:
        Bx_new,By_new,Bz_new,Ex_new,Ey_new,Ez_new,Ux_new,Uy_new,Uz_new \
        = self.find_field_of_grid(Bx,By,Bz,Ex,Ey,Ez,Ux,Uy,Uz, Bx,By,Bz,Ex,Ey,Ez,Ux,Uy,Uz)

        # Update position of particles
        self.find_new_position(Ux[:,:],Uz[:,:])
        self.re_distribution_on_grid()

        # Run the iteration to solve the problem
        # While time is smaller than the maximum time this will continuous running
        while time < self.t_max :

            # When our time is greater than the user provided output time,
            # save fields for output later.
            if time >= next_out:
                print(time)
                self.store_fields(Bx_new,By_new,Bz_new,Ex_new,Ey_new,Ez_new,Ux_new,Uy_new,Uz_new)

                next_out += self.dt_out

            # Set current time step as old time step
            Bx_old,By_old,Bz_old,Ex_old,Ey_old,Ez_old,Ux_old,Uy_old,Uz_old = Bx,By,Bz,Ex,Ey,Ez,Ux,Uy,Uz
            # And new time step as current
            Bx,By,Bz,Ex,Ey,Ez,Ux,Uy,Uz = Bx_new,By_new,Bz_new,Ex_new,Ey_new,Ez_new,Ux_new,Uy_new,Uz_new

            # Calculate new time step form old and current fields
            Bx_new,By_new,Bz_new,Ex_new,Ey_new,Ez_new,Ux_new,Uy_new,Uz_new \
            = self.find_field_of_grid(Bx_old,By_old,Bz_old,Ex_old,Ey_old,Ez_old,Ux_old,Uy_old,Uz_old,
                                        Bx,By,Bz,Ex,Ey,Ez,Ux,Uy,Uz)

            # Update position from current velocities
            self.find_new_position(Ux[:,:],Uz[:,:])
            self.re_distribution_on_grid()
            time += self.dt

        # Save the last time step
        print("Last Time: " + str(time))
        self.store_fields(Bx_new,By_new,Bz_new,Ex_new,Ey_new,Ez_new,Ux_new,Uy_new,Uz_new)


        # Return all desired fields and parameters.
        return np.arange(0,self.Lx,self.dx), np.arange(0,self.Lz,self.dz), self.position_x_stored, self.position_z_stored,\
            self.Ux_stored, self.Uy_stored,self.Uz_stored, self.Bx_stored, self.By_stored, self.Bz_stored, self.Ex_stored, self.Ey_stored, self.Ez_stored


In [ ]:
A = Particle_in_cell_2D(Lx=0.1,Lz=1,Nx=10,Nz=100,t_max=1.7, dt=0.01, dt_out = 0.1, alpha=1,D=1)

x, z, pos_x, pos_z, Ux, Uy, Uz, Bx, By, Bz, Ex, Ey, Ez = A.solve()

In [ ]:
%matplotlib inline
plt.figure(figsize = (15,10))

ax = plt.subplot(4,3,1)
contour1 = ax.contourf(Ux[0][:,:],100)
ax.set_title("Initial Ux")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")
plt.colorbar(contour1, ax=ax)

ax = plt.subplot(4,3,2)
contour1 = ax.contourf(Bx[0][:,:],100)
ax.set_title("Initial Bx")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")
plt.colorbar(contour1, ax=ax)

ax = plt.subplot(4,3,3)
ax.scatter(pos_x[0], pos_z[0], s = 0.8)
ax.set_title("Initialized particle position")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")

ax = plt.subplot(4,3,4)
contour1 = ax.contourf(Ux[6][:,:],100)
ax.set_title("Ux at t = 2.5")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")
plt.colorbar(contour1, ax=ax)

ax = plt.subplot(4,3,5)
contour1 = ax.contourf(Bx[6][:,:],100)
ax.set_title("Bx at t = 2.5")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")
plt.colorbar(contour1, ax=ax)

ax = plt.subplot(4,3,6)
ax.scatter(pos_x[6], pos_z[6], s = 0.8)
ax.set_title("particle position at t = 2.5")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")

ax = plt.subplot(4,3,7)
contour1 = ax.contourf(Ux[12][:,:],100)
ax.set_title("Ux at t = 7.5")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")
plt.colorbar(contour1, ax=ax)

ax = plt.subplot(4,3,8)
contour1 = ax.contourf(Bx[12][:,:],100)
ax.set_title("Bx at t = 7.5")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")
plt.colorbar(contour1, ax=ax)

ax = plt.subplot(4,3,9)
ax.scatter(pos_x[12], pos_z[12], s = 0.8)
ax.set_title("particle position at t = 7.5")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")

ax = plt.subplot(4,3,10)
contour1 = ax.contourf(Ux[-1][:,:],100)
ax.set_title("Ux at t = 10")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")
plt.colorbar(contour1, ax=ax)

ax = plt.subplot(4,3,11)
contour1 = ax.contourf(Bx[-1][:,:],100)
ax.set_title("Bx at t = 10")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")
plt.colorbar(contour1, ax=ax)

ax = plt.subplot(4,3,12)
ax.scatter(pos_x[-1], pos_z[-1], s = 0.8)
ax.set_title("particle position at t = 10")
ax.set_xlabel("Normalized x")
ax.set_ylabel("Normalized z")

plt.tight_layout()
plt.show()

plt.plot(z,Ux[-1][0,:])
plt.xlabel('z')
plt.ylabel('Ux')
plt.title('Ux Breakdown')
plt.show()